In [1]:
import asyncio
import nest_asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from autogen_agentchat.agents import UserProxyAgent, AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination, ExternalTermination
import backoff
import openai

from autogen_core.tools import FunctionTool
from typing_extensions import Annotated

from playwright.async_api import async_playwright
from serpapi import GoogleSearch
import re
import requests
from bs4 import BeautifulSoup
from statistics import median
from typing import List, Dict, Annotated
serp_api_key = '4ce6ea7723c429b154ec8c606d9ca33ae656561a940a61a3c98fdea97d1aa675'


In [2]:
# OpenAI
gpt_model = "gpt-4o-mini"
deepseek_model = "deepseek-reasoner"


In [ ]:
# @backoff.on_exception(backoff.expo, openai.RateLimitError)
def completions_with_backoff(*args, **kwargs):
    return OpenAIChatCompletionClient(model=gpt_model)


async def budget_calculator(costs: Annotated[list[int], "The costs of activities, accomodations and flights."]) -> int:
    # Returns a sum of costs of the activities, accomodations and flights.
    return sum(costs)

def search_booking_info(query: str) -> dict:
    """Search for booking info (activity/hotel/flight) and return name, link, cost, and note."""
    search = GoogleSearch({
        "q": query,
        "api_key": serp_api_key,
        "hl": "en",
        "gl": "us"
    })
    results = search.get_dict()

    booking_info = {
        "activity": query,
        "link": "",
        "cost": "",
        "note": "No relevant link found"
    }

    top_link = ""
    for result in results.get("organic_results", []):
        link = result.get("link", "")
        if any(domain in link for domain in [
            "viator.com", "getyourguide.com", "booking.com",
            "expedia.com", "trip.com", "kayak.com", "tripadvisor.com",
        ]):
            top_link = link
            break

    if not top_link:
        return booking_info

    try:
        response = requests.get(top_link, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text(" ", strip=True)


        prices = re.findall(r'[$€£]\s?(\d{1,4})', text)
        prices = [int(p) for p in prices]

        if prices:
            prices.sort()
            med = median(prices)
            booking_info["cost"] = f"${min(prices)}–${max(prices)}"
            booking_info["note"] = f"Estimated from {len(prices)} prices, median ≈ ${int(med)}"
        else:
            booking_info["note"] = "No prices found on page"

        booking_info["link"] = top_link
        return booking_info

    except Exception as e:
        booking_info["note"] = f"Error scraping: {str(e)}"
        return booking_info

search_tool = FunctionTool(
    name="search_booking_info",
    func=search_booking_info,
    description="Use this to perform necessary web searches. "
)

async def get_playwright():
    playwright = await async_playwright().start()
    return playwright

# @backoff.on_exception(backoff.expo, openai.RateLimitError)
# async def run_stream_with_backoff(team, task):
#     run = asyncio.create_task(Console(team.run_stream(task=task)))
#     await asyncio.sleep(0.1)


# @backoff.on_exception(backoff.expo, openai.RateLimitError)
async def example_usage() -> None:
    model_client = OpenAIChatCompletionClient(model=gpt_model)
    # 配置模型信息
    # model_info = {
    #     "name": "deepseek-chat", # 模型名称，可随意填写
    #     "parameters": {
    #         "max_tokens": 2048,  # 每次输出最大token数
    #                             # deepseek官方数据：1个英文字符 ≈ 0.3 个 token。1 个中文字符 ≈ 0.6 个 token。
    #         "temperature": 0.4,  # 模型随机性参数，数字越大，生成的结果随机性越大，一般为0.7，
    #                             # 如果希望AI提供更多的想法，可以调大该数字
    #         "top_p": 0.9,  # 模型随机性参数，接近 1 时：模型几乎会考虑所有可能的词，只有概率极低的词才会被排除，随机性也越强；
    #                     # 接近 0 时：只有概率非常高的极少数词会被考虑，这会使模型的输出变得非常保守和确定
    #     },
    #     "family": "gpt-4o",  # 必填字段，model属于的类别
    #     "functions": [],  # 非必填字段，如果模型支持函数调用，可以在这里定义函数信息
    #     "vision": False,  # 必填字段，模型是否支持图像输入
    #     "json_output": True,  # 必填字段，模型是否支持json格式输出
    #     "function_calling": True  # 必填字段，模型是否支持函数调用，如果模型需要使用工具函数，该字段为true
    # }

    # # 创建模型
    # model_client = OpenAIChatCompletionClient(model="deepseek-chat", # 必须与官方给的模型名称一致
    #                                         base_url="https://api.deepseek.com", # 调用API地址
    #                                         api_key="sk-8b1dd0297d7547209b3acb703f969c71",
    #                                         model_info=model_info)

    # Create agents
    agents = []
    user = UserProxyAgent(
            name="User",
            description="The user agent.",
            )
    agents.append(user)

    attraction_idea_agent = AssistantAgent(
        name="AttractionRecommender",
        model_client=model_client,
        description="Recommend top attractions for a given destination using general knowledge only, no web search. Include a short description of each attraction")
    agents.append(attraction_idea_agent)

    budget_checker_agent = AssistantAgent(
        name="BudgetController",
        model_client=model_client,
        description="Compare total travel costs to the user-defined budget and say whether it fits.")
    agents.append(budget_checker_agent)


    AttractionLink_agent = AssistantAgent(
        name="AttractionLinkFinder",
        model_client=model_client,
        tools=[search_tool],
        description='''Using the search tool, return a link for every attraction (from the approved attraction list).
        Post all links to the orchestrator''')
    agents.append(AttractionLink_agent)

    HotelLink_agent = AssistantAgent(
        name="HotelLinkFinder",
        model_client=model_client,
        tools=[search_tool],
        description='''Using the search tool, return a link for a hotel from the destination according to the chosen price range.
        Post all links to the orchestrator''')
    agents.append(HotelLink_agent)

    FlightLink_agent = AssistantAgent(
        name="FlightLinkFinder",
        model_client=model_client,
        tools=[search_tool],
        description='''Using the search tool, return a link for flights to/from the destination according to the chosen price range.
        Post all links to the orchestrator''')
    agents.append(FlightLink_agent)


    playwright = await get_playwright()
    surfer1 = MultimodalWebSurfer(
            name="ActivityPricer",
            model_client=model_client,
            #description="A web surfer agent that search for things to do in the destination. It also need to return the booking fee for each activity.",
            description = '''
            You are a web agent looking for pricing information on. 
            
            You need to use web_search to find the web page of activities, hotels and flights, use summarize_page to summarize the page and extract the price information, and use sleep in the end each time.

            Return the prices found in this format:
            {
            "estimated_price": "$30-40",
            "notes": "Price for standard ticket. Based on visible listings on the page."
            }

            Do not return other descriptions or ratings unless price is unavailable. Only extract from visible page content.

            ''',
            playwright=playwright,
        )
    agents.append(surfer1)

    schedule_maker = AssistantAgent(
            name="ScheduleMaker",
            model_client = model_client,
            description="""You are a schedule maker and have extensive knowledge of travel route and map.
            Based on the activities and destinations, create a daily itinerary:
            1. Optimize routes between locations
            2. Include travel times
            3. Balance activities per day"""
        )
    agents.append(schedule_maker)

    # Create a group chat
    cond1 = TextMentionTermination("TERMINATE")
    external_termination = ExternalTermination()
    team = MagenticOneGroupChat(agents, model_client=model_client, termination_condition=cond1 | external_termination)

    task = """
    Task: Plan a Travel Itinerary

    You will generate a detailed travel itinerary including the following information:
    - Place of Departure (city, country)
    - Destination
    - Travel Dates (schedule)
    - Accommodation
    - Activities (things to do)
    - Budget

    Use the Chain-of-Thought method by completing each task step-by-step, explicitly reasoning at each step. Provide a few-shot example for guidance.

    Step-by-Step Instruction:

    Step 1: Mandatory Information Collection
    First, ask the user explicitly:
    "Please tell me your place of departure (city and country). This is mandatory information to start planning your itinerary."

    Reasoning: The itinerary cannot be planned without knowing the starting location. Ensure the user provides this information clearly.

    Few-shot Example:
    Agent: Please tell me your place of departure (city and country). This is mandatory information.
    User: Boston, USA

    Step 2: Collect Additional Travel Information (with Recommendations)
    Next, sequentially ask the user for destination, travel dates, and expected budget. When asking each question, always provide 3 recommendation options.

    Reasoning: Providing options helps users make quicker and informed decisions.

    Few-shot Example:
    Agent: Where would you like to travel? Here are three popular recommendations:
    1. San Francisco, USA
    2. New York City, USA
    3. Miami, USA
    User: I'd like to go to San Francisco, USA.

    Agent: Great choice! What dates are you planning for your travel? Here are three recommended timeframes:
    1. May 20 - May 27
    2. June 10 - June 17
    3. July 15 - July 22
    User: June 10 - June 17

    Agent: What's your expected budget for the entire trip? Here are three typical budget ranges:
    1. Economy: $1,000 - $1,500
    2. Moderate: $1,500 - $2,500
    3. Luxury: $2,500+
    User: Moderate, around $2,000

    Step 3: Generate Activities and Accommodations
    Now, you must use the AttractionRecommender to produce a list of attractions the user approves.
    Then, give the approved list to the AttractionLinkFinder and request it to search for them one by one, only accept the search
    results as links. Save the links. Then, using the destination, ask the HotelLinkFinder to find appropriate accomodations and flights links
    Only accept the name and link of the results. Save the links and pass flight links to FlightLinkFinder, pass hotel links to
    finder for flights. Ask ActivityPricer about the price and go through it one by one.
    Save all information regarding pricing and pass to BudgetController.

    Reasoning: To provide accurate recommendations, you need to align activities and accommodations with the user's budget and interests.

    Few-shot Example:
    Agent Search Criteria:
    - Destination: San Francisco, USA
    - Dates: June 10 - June 17
    - Budget: Moderate (~$2,000)
    - Activities: Popular tourist spots, cultural experiences
    - Accommodations: Mid-range hotels or Airbnb

    Sample Agent Output:
    Activities:
    1. Visit Golden Gate Bridge (Free)
    2. Alcatraz Island Tour ($40)
    3. Exploratorium ($30)

    Accommodation Recommendation:
    1. Holiday Inn Golden Gateway ($180/night)
    2. Airbnb near Fisherman's Wharf ($150/night)

    Would you like to proceed with this plan? (yes/no)
    User: Yes

    Step 4: Calculate and Present Budget Clearly
    Calculate the total expected cost based on accommodation, activities, transportation, and daily expenses clearly.

    Reasoning: Users appreciate transparency and a clear breakdown of expenses.

    Few-shot Example:
    Budget Breakdown:
    - Flights (Round-trip): $400
    - Accommodation (7 nights x $150): $1,050
    - Activities: $70
    - Meals and Miscellaneous: $400
    - Total: ~$1,920
    Would you like to proceed with this plan? (yes/no)
    User: Yes

    Step 5: Final Travel Plan Summary
    Provide a concise and comprehensive summary of the travel itinerary, ask if the user approve of the summary, then send to schedule making agent for final itinerary.

    Reasoning: Summaries help users quickly understand and confirm the overall plan.

    Few-shot Example:
    Agent:
    Here is your travel itinerary summary:
    - Departure: Boston, USA
    - Destination: San Francisco, USA
    - Dates: June 10 - June 17
    - Accommodation: Airbnb near Fisherman's Wharf
    - Activities: Golden Gate Bridge, Alcatraz Tour, Exploratorium
    - Total Budget: ~$1,920
    Would you like to proceed with this plan? (yes/no)
    User: Yes

    Step 6:
    Show final itinerary for user approval.
    Reasoning - the final result should align with the user's preferences
    """

    stream = team.run_stream(task=task)
    await Console(stream)
    await cond1.reset()    

    # run_stream_with_backoff(team, task)
    # run = asyncio.create_task(Console(team.run_stream(task=task)))
    # await asyncio.sleep(0.1)
    # external_termination.set()
    # await run

    # while True:
    #     try: 
    #         await Console(team.run_stream(task="Continue the task if not terminated. "))
    #         external_termination.set()
    #         await run
    #     except openai.RateLimitError as e:
    #         print(f"Rate limit error: {e}")
    #         await asyncio.sleep(5)
    #     except ValueError as e:
    #         print(f"Value error: {e}")
    #         await asyncio.sleep(5)



In [14]:
if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(example_usage())

---------- user ----------

    Task: Plan a Travel Itinerary

    You will generate a detailed travel itinerary including the following information:
    - Place of Departure (city, country)
    - Destination
    - Travel Dates (schedule)
    - Accommodation
    - Activities (things to do)
    - Budget

    Use the Chain-of-Thought method by completing each task step-by-step, explicitly reasoning at each step. Provide a few-shot example for guidance.

    Step-by-Step Instruction:

    Step 1: Mandatory Information Collection
    First, ask the user explicitly:
    "Please tell me your place of departure (city and country). This is mandatory information to start planning your itinerary."

    Reasoning: The itinerary cannot be planned without knowing the starting location. Ensure the user provides this information clearly.

    Few-shot Example:
    Agent: Please tell me your place of departure (city and country). This is mandatory information.
    User: Boston, USA

    Step 2: Collec

In [ ]:
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from typing_extensions import Annotated


async def budget_calculator(costs: Annotated[list[int], "The costs of activities, accomodations and flights."]) -> int:
    # Returns a sum of costs of the activities, accomodations and flights.
    return sum(costs)


# Create a function tool.
stock_price_tool = FunctionTool(budget_calculator, description="Get the sum of costs.")

# Run the tool.
cancellation_token = CancellationToken()
result = await stock_price_tool.run_json({"ticker": "AAPL", "date": "2021/01/01"}, cancellation_token)

# Print the result.
print(stock_price_tool.return_value_as_string(result))

In [ ]:
llm_config = {
    "model": gpt_model
}

# create an AssistantAgent instance named "assistant"
assistant = AssistantAgent(
    name="assistant",
    model_client=OpenAIChatCompletionClient(**llm_config),
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = UserProxyAgent(
    name="user_proxy"
)
agent_task = asyncio.create_task(
            user_proxy.on_messages(
                [TextMessage(content="What is your name? ", source="user")],
                cancellation_token=token,
            )
        )
        response = await agent_task
        assert isinstance(response.chat_message, TextMessage)
        print(f"Your name is {response.chat_message.content}")

TypeError: AssistantAgent.__init__() got an unexpected keyword argument 'llm_config'